# Легкий способ парсинга новостных статей на Python

В современном цифровом мире нас засыпают бесконечным потоком информации.
Мы постоянно прокручиваем социальные сети и 24 часа в сутки имеем доступ к новостным каналам.
Таким образом, существует множество новостей, которые нам надо знать и мы должны быть в состоянии их все быстро переварить.

Итак, давайте разберем упражнение по сжатию новостных статей до размера, более удобного для их восприятия.

Мы спарсим примерную статью, используя библиотеки request и BeautifulSoup, а затем сформируем ее краткое изложение при помощи великолепной библиотеки gensim.

In [10]:
# Импорт необходимых библиотек
# Pandas & Numpy
# import pandas as pd
# import numpy as np
# import sys

import requests
from bs4 import BeautifulSoup
from gensim.summarization import summarize

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 80 from PyObject

In [8]:
# MISTAKE !!!!
# ModuleNotFoundError: No module named 'numpy.random.bit_generator'

# Запуск следующей строки заставил переустановить numpyпакет.
# Поскольку пакет был каким-то образом поврежден, это было исправлено.
# conda install numpy --force-reinstall

# Или можно попробовать так:
# pip3 install scipy==1.7.1 numpy==1.18.5 scikit-learn==0.24.2 --no-cache-dir --no-binary :all:

!pip install numpy --force-reinstall

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'C:\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [9]:
# Установка библиотеки для более удобного для их восприятия текстовых документов и статей
# !pip install gensim

#### Статья по которой воспроизводится данный урок лежит по ссылке:

https://pythonist.ru/legkij-sposob-parsinga-novostnyh-statej-na-python/

In [ ]:
# Теперь выберем интересную статью:
# Получаем текст страницы
url = 'https://www.npr.org/2019/07/10/740387601/university-of-texas-austin-promises-free-tuition-for-low-income-students-in-2020'
page = requests.get(url).text
# print(page)

## Вебскрейпинг
### Теперь приступим непосредственно к вебскрейпингу!

In [ ]:
# Сначала мы превратим содержимое страницы в объект BeautifulSoup, что позволит нам анализировать HTML-теги.
# Turn page into BeautifulSoup object to access HTML tags
soup = BeautifulSoup(page)

Затем нам нужно выяснить, какие HTML-теги содержат заголовок и основной текст статьи.
В качестве отличного учебника по HTML можно использовать сайт HTML.com.

[python_ad_block]
Чтобы это сделать, мы будем использовать инструменты разработчика Google Chrome.
Откроем статью в новой вкладке, кликнем по ней правой кнопкой мыши и в выпавшем меню выберем пункт Inspect (Просмотр кода).
Это вызовет DevTools (инструменты разработчика) в панели справа:

Чтобы найти все HTML-теги, соответствующие всему, что вы видите на странице, нажмите на небольшую кнопочку наверху.
На картинке она отмечена синей стрелкой.

Теперь наводим указатель мыши на фрагмент страницы, который мы хотим исследовать.
В данном случае это заголовок и основной текст статьи. И мы видим, какие теги отвечают за формат данного текста.

In [ ]:
# Заголовок статьи окружен c двух сторон тегом <h1>. Загрузим этот фрагмент следующим образом:

# Get headline
headline = soup.find('h1').get_text()

In [ ]:
# Основной текст статьи окружен тегами <p>. На этот раз нам нужно будет найти все такие теги,
# содержащиеся на странице, поскольку в них заключен каждый абзац статьи.

# Get text from all <p> tags.
p_tags = soup.find_all('p’)
# Get the text from each of the “p” tags and strip surrounding whitespace.
p_tags_text = [tag.get_text().strip() for tag in p_tags]

Если вы изучите текст, который мы сохранили в переменную p_tags_text, то заметите, что есть некоторые фрагменты текста, не относящиеся к основной статье, например имя автора и некоторые подписи к изображениям.
Они также заключены в теги <p>, поэтому и оказались здесь.
Чтобы очистить этот текст от фрагментов, которые не являются частью основной статьи, мы в качестве фильтров используем несколько представлений списков.

В этой статье подписи к изображениям содержат символ новой строки \n.
Поскольку мы знаем, что в настоящих предложениях в статье не бывает случайных переносов строк, мы можем смело отказаться от таких фрагментов.
Точно так же мы можем отбрасывать фрагменты текста, не содержащие точки, поскольку мы знаем, что любое правильное предложение в статье будет содержать точку.
Так мы сможем отбросить имя автора и некоторые другие ненужные нам вещи.

In [ ]:
# 
# Filter out sentences that contain newline characters '\n' or don't contain periods.
sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
sentence_list = [sentence for sentence in sentence_list if '.' in sentence]
# Combine list items into string.
article = ' '.join(sentence_list)

## Резюме статьи

Теперь, имея полный текст статьи, мы хотим получить его резюме (краткий пересказ содержания статьи).

Gensim — отличный пакет Python для большого количества задач нейролингвистического программирования (НЛП).
Он включает в себя довольно надежную функцию резюмирования, которой достаточно легко пользоваться.
Она реализует разновидность алгоритма TextRank.

In [ ]:
# Для использования этой функции нам нужна лишь одна строчка кода:

summary = summarize(article_text, ratio=0.3)

## Вот и все, мы сделали, что хотели!

### Мы извлекли заголовок статьи и получили краткое изложение ее содержания.

#### Теперь вы можете понять суть статьи примерно в три раза быстрее и сэкономить время для других дел. 

In [ ]:
# 


In [ ]:
# 


In [ ]:
# 


In [ ]:
# 


In [ ]:
# 


In [ ]:
# 


In [ ]:
# 


In [ ]:
# 
